# **Sentiment Analysis for Social Media Data**

----

In [1]:
import tensorflow as tf; 
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
%pip install asyncpraw
%pip install nest-asyncio
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for asyncpraw from https://files.pythonhosted.org/packages/c4/34/a90bf7476e8c1dd5163e37a0a29164c930b69463cb2d1748fde0afcddf40/asyncpraw-7.7.1-py3-none-any.whl.metadata
  Using cached asyncpraw-7.7.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiofiles-0.8.0-py3-none-any.whl (13 kB)
  Using cached aiosqlite-0.17.0-py3-none-any.whl (15 kB)
  Obtaining dependency information for asyncprawcore<3,>=2.1 from https://files.pythonhosted.org/packages/1d/a0/a1f216c7ad7fd94482242eb92d6903c0ed5f3340436b952db2a42d2eddeb/asyncprawcore-2.4.0-py3-none-any.whl.metadata
  Using cached asyncprawcore-2.4.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
Using cached asyncpraw-7.7.1-py3-none-any.whl (196 kB)
Using cached asyncprawcore-2.4.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ypy-websocket 0.8.2 requires aiofiles<23,>=22.1.0, but you have aiofiles 0.8.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install transformers
# %pip install numpy

^C
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import asyncio
import asyncpraw
import pandas as pd
import nest_asyncio 
import os
import warnings
import torch
from datasets import Dataset
from transformers import pipeline

nest_asyncio.apply()

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
reddit = asyncpraw.Reddit(
    client_id='xxxxxx',
    client_secret='xxxxxx',
    user_agent='Islam_final_project',
)

In [5]:
output_folder = os.path.join( os.getcwd(), "content")
output_file = os.path.join( output_folder, "reddit_comments.csv")

related_subreddits = ['politics']
async def fetch_comments(subreddit_names, num_comments_per_topic, total_comments, batch_size, output_file, delay_threshold=100, delay_seconds=5):

    data = {'Title': [], 'Comment': [], 'Upvote Ratio': []}
    comments_count = 0

    for subreddit_name in subreddit_names:
        subreddit = await reddit.subreddit(subreddit_name)
        
        async for submission in subreddit.hot(limit=None):  
            await submission.load()
            submission.comment_sort = 'top'
            async for comment in submission.comments:
                if isinstance(comment, asyncpraw.models.reddit.more.MoreComments):
                    continue

                data['Title'].append(submission.title)
                data['Comment'].append(comment.body)
                data['Upvote Ratio'].append(submission.upvote_ratio)
                comments_count += 1
                if comments_count >= total_comments:
                    break
                if comments_count % batch_size == 0:
                    print(f' Progress = {comments_count} : {total_comments}')
                    write_to_csv(data, output_file)
                    data = {'Title': [], 'Comment': [], 'Upvote Ratio': []}
                if comments_count % delay_threshold == 0:
                    await asyncio.sleep(delay_seconds)

            if comments_count >= total_comments:
                break

    if data['Title']:
        write_to_csv(data, output_file)

def write_to_csv(data, output_file):
    df = pd.DataFrame(data)
    df.to_csv(output_file, mode='a', index=False, header=not pd.DataFrame().empty)

async def main():
    num_comments_per_topic = 10 
    total_comments = 100
    batch_size = 1000

    delay_threshold = 100 
    delay_seconds = 5

    await fetch_comments(related_subreddits, num_comments_per_topic, total_comments, batch_size, output_file, delay_threshold, delay_seconds)

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


In [6]:
df = pd.read_csv(output_file,names=['Title', 'Comment', 'upRatio'])
print(df.columns)
df

Index(['Title', 'Comment', 'upRatio'], dtype='object')


,Title,Comment,upRatio
0,Christie to Ramaswamy: ‘So Shut Up for a Littl...,Please play the race card. I love it when Rep...,0.90
1,Christie to Ramaswamy: ‘So Shut Up for a Littl...,RamaSmarmy,0.90
2,"Targeting costly meds, Biden admin asserts aut...","\nAs a reminder, this subreddit [is for civil ...",0.98
3,"Targeting costly meds, Biden admin asserts aut...",I would recommend the government require being...,0.98
4,"Targeting costly meds, Biden admin asserts aut...",This is a bluff to get big pharma to back down...,0.98
...,...,...,...
13095,Republicans Rage After Hunter Biden Snubs Subp...,He was there to take questions he just wanted ...,0.93
13096,Republicans Rage After Hunter Biden Snubs Subp...,I like my guys with two impeachments and 90+fe...,0.93
13097,Republicans Rage After Hunter Biden Snubs Subp...,As always the best response to a republican is...,0.93
13098,Republicans Rage After Hunter Biden Snubs Subp...,How many subpoenas did Trump & Idiots basicall...,0.93


In [7]:
classifier = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")
def get_sentiment(text):
    result = classifier(text, truncation=True)
    return result[0]['label']
df['Sentiment'] = df['Comment'].apply(get_sentiment)

---
---

In [8]:
output_sentiment_file = os.path.join( output_folder, "reddit_comments_with_sentiment.csv")
df.to_csv(output_sentiment_file, index=False)

In [9]:
df = pd.read_csv(output_sentiment_file)
print(df.columns)
df

Index(['Title', 'Comment', 'upRatio', 'Sentiment'], dtype='object')


,Title,Comment,upRatio,Sentiment
0,Christie to Ramaswamy: ‘So Shut Up for a Littl...,Please play the race card. I love it when Rep...,0.90,positive
1,Christie to Ramaswamy: ‘So Shut Up for a Littl...,RamaSmarmy,0.90,positive
2,"Targeting costly meds, Biden admin asserts aut...","\nAs a reminder, this subreddit [is for civil ...",0.98,negative
3,"Targeting costly meds, Biden admin asserts aut...",I would recommend the government require being...,0.98,positive
4,"Targeting costly meds, Biden admin asserts aut...",This is a bluff to get big pharma to back down...,0.98,negative
...,...,...,...,...
13095,Republicans Rage After Hunter Biden Snubs Subp...,He was there to take questions he just wanted ...,0.93,negative
13096,Republicans Rage After Hunter Biden Snubs Subp...,I like my guys with two impeachments and 90+fe...,0.93,negative
13097,Republicans Rage After Hunter Biden Snubs Subp...,As always the best response to a republican is...,0.93,negative
13098,Republicans Rage After Hunter Biden Snubs Subp...,How many subpoenas did Trump & Idiots basicall...,0.93,negative


In [10]:
df['Sentiment'].value_counts()

Sentiment
negative    9309
positive    3089
neutral      702
Name: count, dtype: int64